## Efficient Fish Acute Toxicity Prediction Using QSAR Machine Learning Pipeline

# 1. SMILES
### Dataset
<br>https://qsardb.org/repository/handle/10967/195</br>
<br>Property pLC50: 96-h acute fish toxicity as log(1/LC50) [-log(mmol/L)]</br>
<br>Training set</br>

In [ ]:
!wget https://qsardb.org/repository/bitstream/handle/10967/195/2015SQER301.qdb.zip
!unzip 2015SQER301.qdb.zip

In [ ]:
files = !ls compounds/
files = [f'compounds/{i}/daylight-smiles' for i in files[1:]]
files

In [ ]:
smi_dict = {}
for file in files:
    f = open(file, 'r')
    smi = f.readline()
    smi_dict[file.split('/')[1]] = smi
    f.close()
smi_dict

In [ ]:
import pandas as pd
import numpy as np
import pubchempy as pcp

In [ ]:
df = pd.read_csv('properties/pLC50/values', sep='\t')
df

In [ ]:
df['SMILES'] = np.nan
for i in range(df.shape[0]):
    try:
        df['SMILES'].iloc[i] = smi_dict[df['Compound Id'].iloc[i]]
    except:
        continue

df = df.dropna().reset_index(drop=True)
df.columns = ['ID', 'pLC50', 'SMILES']
target = df[['pLC50']]
del df['pLC50']
df

In [ ]:
f = open('compounds/compounds.xml', 'r')
lines = f.readlines()
f.close()

In [ ]:
df['InChI'] = 'Miss'
for i in range(len(lines)):
    if 'Tab' in lines[i]:
        if lines[i][12:-6] in df['ID'].tolist():
            if 'InChI'in lines[i+4]:
                key = lines[i+4][15:-9]
            if 'InChI'in lines[i+5]:
                key = lines[i+5][15:-9]
            if 'InChI'in lines[i+6]:
                key = lines[i+6][15:-9]
            n = df[df['ID'] == lines[i][12:-6]].index[0]
            df['InChI'].iloc[n] = key
            key = 'Miss'

df

In [ ]:
df['Name'] = 'Miss'
for i in range(len(lines)):
    if 'Tab' in lines[i]:
        if lines[i][12:-6] in df['ID'].tolist():
            if 'Name'in lines[i+1]:
                key = lines[i+1][14:-8]
            if 'Name'in lines[i+2]:
                key = lines[i+2][14:-8]
            n = df[df['ID'] == lines[i][12:-6]].index[0]
            df['Name'].iloc[n] = key
            key = 'Miss'
df

In [ ]:
for i in range(df.shape[0]):
    if df['InChI'].iloc[i] != 'Miss':
        compound = pcp.get_compounds(df['InChI'].iloc[i], 'inchi')
        # print(compound)
        if len(str(compound[0])) > 10:
            df['ID'].iloc[i] = compound[0].cid
        else:
            df['ID'].iloc[i] = 'Miss'
            print(i)
        
# df = df.dropna().reset_index(drop=True)
df

In [ ]:
for i in range(df.shape[0]):
    if df['ID'].iloc[i] == 'Miss':
        if df['Name'].iloc[i] != 'Miss':
            compound = pcp.get_compounds(df['Name'].iloc[i], 'name')
            # print(compound)
            if len(str(compound[0])) > 10:
                df['ID'].iloc[i] = compound[0].cid
            else:
                df['ID'].iloc[i] = np.nan
                print(i)
        
df = df.dropna().reset_index(drop=True)
df

In [ ]:
df['SMILES'] = np.nan
for i in range(df.shape[0]):
    try:
        c = pcp.Compound.from_cid(df['ID'].iloc[i])
        df['SMILES'].iloc[i] = c.canonical_smiles
    except:
        print(i)
        continue
df

In [ ]:
df = df[['ID', 'SMILES']].dropna().reset_index(drop=True)
df.columns = ['CID', 'SMILES']
df

# 2. RDKit
### Canonicalization

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from mordred import Calculator, descriptors
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def canonical(df):
    for i in range(df.shape[0]):
        mol = Chem.MolFromSmiles(df['SMILES'].iloc[i])
        if mol is not None:
            smi = Chem.MolToSmiles(mol)
            df['SMILES'].iloc[i] = smi
        else:
            df['SMILES'].iloc[i] = np.nan
            print(i)
    df = df.dropna().reset_index(drop=True)
    return df

In [ ]:
df = canonical(df)
df

In [ ]:
df.to_csv('Train.csv', index=False)

# 3. Mordred
### Descriptors

In [ ]:
def mordred(df):
    calc = Calculator(descriptors, ignore_3D=True)
    mols = [Chem.MolFromSmiles(s) for s in df['SMILES'].tolist()]
    mord = calc.pandas(mols)
    df = pd.concat([df, mord], axis=1)
    return df

In [ ]:
df = mordred(df)
df

In [ ]:
def invalid(df):
    for c in df.columns:
        if c not in ['CID', 'SMILES']:
            for i in range(df.shape[0]):
                try: 
                    _ = float(df[c].iloc[i])+1
                    df[c].iloc[i] = float(df[c].iloc[i])
                except:
                    df[c].iloc[i] = np.nan
                    continue
    return df

In [ ]:
df = invalid(df)
df

In [ ]:
df['pLC50'] = target['pLC50']
df

In [ ]:
df.to_csv('Train_Mordred.csv', index=False)

# 4. PyQSAR
### Preprocessing

In [ ]:
# Python 2.7

In [ ]:
from pyqsar import data_tools as dt
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Train_Mordred.csv')
df = df.fillna(0)
X_data = df.iloc[:,2:-1]
y_data = df.iloc[:,-1:]

X_data = dt.rm_empty_feature(X_data)
X_data = dt.rmNaN(X_data)

Infinity_list = []
for des in X_data:
    my_list = X_data[des].values
    for val in my_list:
        if val == "Infinity" or val == "-Infinity":
            Infinity_list.append(des)
            del X_data[des]
            break

header = list(X_data.columns.values)
scaler = preprocessing.MinMaxScaler()
X_data = scaler.fit_transform(X_data)
X_data = pd.DataFrame(X_data, columns=header)

df = pd.concat([df[['CID', 'SMILES']], X_data, y_data], axis=1)
df.to_csv('Train_Mordred_Prepro.csv', index=False)
df

In [ ]:
df = pd.read_csv('Train_Mordred_Prepro.csv')

In [ ]:
kwargs = dict(hist_kws={'alpha':.3}, kde_kws={'linewidth':2})

plt.figure(figsize=(8,5))#, dpi= 500)
sns.distplot(df['pLC50'], color="blue", bins=16, **kwargs)
plt.xlabel(r'$pLC_{50}$', fontsize=25)
plt.xticks(fontsize=18)
plt.ylabel('Density', fontsize=25)
plt.yticks(fontsize=18)
plt.show()

# 5. PyCaret
### Comparison & Tuning

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from pycaret.regression import *

In [ ]:
df = pd.read_csv('Train_Mordred_Prepro.csv')
df = df.fillna(0)
df

In [ ]:
exp = setup(df, target='pLC50', ignore_features = ['CID', 'SMILES'], silent=True)

In [ ]:
top3 = compare_models(n_select = 3, sort='r2')

In [5]:
best = top3[0]

NameError: name 'top3' is not defined

In [ ]:
best.get_params()

In [ ]:
tuned_best = tune_model(best, n_iter=50, optimize='R2')

In [ ]:
tuned_best.get_params()

In [ ]:
tuned_best = tune_model(best, n_iter=100, optimize='R2')

In [ ]:
tuned_best.get_params()

# 6. Scikit-learn
### First Test

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [7]:
def prepro(df, select):
    df = df.fillna(0)
    X_data = df.iloc[:,2:-1]
    y_data = df.iloc[:,-1:]
    header = list(X_data.columns.values)
    scaler = preprocessing.MinMaxScaler()
    X_data = scaler.fit_transform(X_data)
    X_data = pd.DataFrame(X_data, columns=header)
    X_data = X_data[select]
    return X_data, y_data

In [8]:
def fit(x, y, ratio=0.2, seed=2021, cv=3):
    
    reg = BayesianRidge(alpha_1=0.0001,
                       alpha_2=0.2,
                       compute_score=True,
                       copy_X=True,
                       fit_intercept=True,
                       lambda_1=0.3,
                       lambda_2=1e-07,
                       n_iter=300,
                       normalize=True,
                       tol=0.001,
                       verbose=False)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=ratio, random_state=seed)
    
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    cv_score = cross_validate(reg, x, y, cv=cv, scoring='r2')

    return reg, r2, mae, mse, y_test, y_pred, cv_score

In [9]:
def plot(r2, mae, mse, y_true, y_pred, cv_score):
    print('%.1i-fold CV R2 : %.4f' % (int(len(cv_score['test_score'])), float(np.mean(cv_score['test_score']))))
    
    plt.figure(figsize=(4, 4))#, dpi=500)
    plt.scatter(y_true, y_pred, alpha=0.4, color='blue')
    plt.plot(y_true, y_true, alpha=0.4, color='black')
    plt.xlabel(r"Measured toxicity ($pLC_{50}$)", fontsize=18)
    plt.ylabel(r"Predicted toxicity ($pLC_{50}$)", fontsize=18)
#     plt.text(-0.6, 3.6, r'$R^2 = 0.9229$', fontdict={'size': 16})
    plt.tight_layout()
    plt.show()
    
    print('R2 : %.4f' % (float(r2)))
    print('MAE : %.4f' % (float(mae)))
    print('MSE : %.4f' % (float(mse)))

In [ ]:
df = pd.read_csv('Train_Mordred_Prepro.csv')
select = df.columns[2:-1]
select

In [ ]:
X_data, y_data = prepro(df, select)
reg, r2, mae, mse, y_test, y_pred, cv_score = fit(X_data, y_data, ratio=0.2, seed=2021, cv=3)
plot(r2, mae, mse, y_test, y_pred, cv_score)

In [ ]:
r2_list = []
mae_list = []
mse_list = []
for i in range(0, 1000, 100):
    print(i)
    X_data, y_data = prepro(df, select)
    reg, r2, mae, mse, y_test, y_pred, cv_score = fit(X_data, y_data, ratio=0.2, seed=i, cv=3)
    plot(r2, mae, mse, y_test, y_pred, cv_score)
    r2_list.append(r2)
    mae_list.append(mae)
    mse_list.append(mse)
print()
print('Total')
print('R2 : %.4f' % (float(np.mean(r2_list))))
print('StD : %.4f' % (float(np.std(r2_list))))
print('MAE : %.4f' % (float(np.mean(mae_list))))
print('MSE : %.4f' % (float(np.mean(mse_list))))

# 8. PyQSAR
### Feature Selection

In [ ]:
# Python 2.7

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pyqsar import clustering as cl
from pyqsar import feature_selection_single as fss

In [ ]:
def fit(x, y, external=False, ratio=0.2, seed=2021, cv=3):
    
    reg = BayesianRidge(alpha_1=0.0001,
                       alpha_2=0.2,
                       compute_score=True,
                       copy_X=True,
                       fit_intercept=True,
                       lambda_1=0.3,
                       lambda_2=1e-07,
                       n_iter=300,
                       normalize=True,
                       tol=0.001,
                       verbose=False)

    
    print(x.shape[1])
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=ratio, random_state=seed)
    
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    cv_score = cross_validate(reg, x, y, cv=cv, scoring='r2')

    return reg, r2, mae, mse, y_test, y_pred, cv_score

In [ ]:
df = pd.read_csv('Train_Mordred_Prepro.csv')
df = df.fillna(0)
X_data = df.iloc[:,2:-1]
y_data = df.iloc[:,-1:]
header = list(X_data.columns.values)
scaler = preprocessing.MinMaxScaler()
X_data = scaler.fit_transform(X_data)
X_data = pd.DataFrame(X_data, columns=header)
X_data

In [ ]:
cl.cophenetic(X_data)
clust = cl.FeatureCluster(X_data, 'average', 2)
clust_info = clust.set_cluster()

In [ ]:
br_select_list = []
br_cv_list = []
for i in range(1, 51):
    print(i)
    select = fss.selection(X_data, y_data,
                           clust_info,
                           model='regression',
                           learning=1000,
                           bank=200,
                           component=i)
    reg, r2, mae, mse, y_test, y_pred, cv_score = fit(X_data[select], y_data, external=False, ratio=0.2, seed=2021, cv=3)
    br_cv_list.append(np.mean(cv_score['test_score']))
    br_select_list.append(select)
    print(select)
    print(np.mean(cv_score['test_score']))
    print

In [ ]:
results = pd.DataFrame({'R2':br_cv_list, 'BR':br_select_list})#, 'RF CV R2':rf_cv_list, 'Rf':rf_select_list})
results = results.sort_values(['R2'], ascending=False).reset_index(drop=True)
results

In [ ]:
results.to_json('PyQSAR_BR.json', orient='table')

# 8. Scikit-learn
### Second Test

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
def prepro(df, select):
    df = df.fillna(0)
    X_data = df.iloc[:,2:-1]
    y_data = df.iloc[:,-1:]
    header = list(X_data.columns.values)
    scaler = preprocessing.MinMaxScaler()
    X_data = scaler.fit_transform(X_data)
    X_data = pd.DataFrame(X_data, columns=header)
    X_data = X_data[select]
    return X_data, y_data

In [ ]:
def fit(x, y, ratio=0.2, seed=2021, cv=3):
    
    reg = BayesianRidge(alpha_1=0.0001,
                       alpha_2=0.2,
                       compute_score=True,
                       copy_X=True,
                       fit_intercept=True,
                       lambda_1=0.3,
                       lambda_2=1e-07,
                       n_iter=300,
                       normalize=True,
                       tol=0.001,
                       verbose=False)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=ratio, random_state=seed)
    
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    cv_score = cross_validate(reg, x, y, cv=cv, scoring='r2')

    return reg, r2, mae, mse, y_test, y_pred, cv_score

In [ ]:
def plot(r2, mae, mse, y_true, y_pred, cv_score):
    print('%.1i-fold CV R2 : %.4f' % (int(len(cv_score['test_score'])), float(np.mean(cv_score['test_score']))))
    
    plt.figure(figsize=(4, 4))#, dpi=500)
    plt.scatter(y_true, y_pred, alpha=0.4, color='blue')
    plt.plot(y_true, y_true, alpha=0.4, color='black')
    plt.xlabel(r"Measured toxicity ($pLC_{50}$)", fontsize=18)
    plt.ylabel(r"Predicted toxicity ($pLC_{50}$)", fontsize=18)
#     plt.text(-0.6, 3.6, r'$R^2 = 0.9229$', fontdict={'size': 16})
    plt.tight_layout()
    plt.show()
    
    print('R2 : %.4f' % (float(r2)))
    print('MAE : %.4f' % (float(mae)))
    print('MSE : %.4f' % (float(mse)))

In [ ]:
df = pd.read_csv('Train_Mordred_Prepro.csv')
select = df.columns[2:-1]
select

In [ ]:
selected = pd.read_json('PyQSAR_BR.json', orient='table')
select = selected['BR'].iloc[0]

selected['Count'] = 0
for i in range(selected.shape[0]):
    selected['Count'].iloc[i] = len(selected['BR'].iloc[i])

selected = selected.sort_values(['Count']).reset_index(drop=True)
max_v = selected[selected['R2'] == max(selected['R2'])]

len(select)

In [ ]:
plt.figure(figsize=(4, 4))#, dpi=500)

plt.subplot(2, 1, 1) 
plt.plot(selected['Count'], selected['R2'], color='blue')
plt.scatter(max_v['Count'], max_v['R2'], color='red')
plt.ylabel(r"$R^2$", fontsize=18)
plt.xlim([-1, 51])
plt.ylim([0, 1])

plt.subplot(2, 1, 2) 
plt.plot(selected['Count'], selected['R2'], color='blue')
plt.plot([0, 50], [0, 0], color='black', alpha=0.4)
plt.xlabel("Number of descriptors", fontsize=18)
plt.ylabel(r"$R^2$", fontsize=18)
plt.xlim([-1, 51])
plt.ylim([-3.5, 1.5])
plt.tight_layout()
plt.show()

In [ ]:
X_data, y_data = prepro(df, select)
reg, r2, mae, mse, y_test, y_pred, cv_score = fit(X_data, y_data, ratio=0.2, seed=2021, cv=3)
plot(r2, mae, mse, y_test, y_pred, cv_score)

In [ ]:
r2_list = []
mae_list = []
mse_list = []
for i in range(0, 1000, 100):
    print(i)
    X_data, y_data = prepro(df, select)
    reg, r2, mae, mse, y_test, y_pred, cv_score = fit(X_data, y_data, ratio=0.2, seed=i, cv=3)
    plot(r2, mae, mse, y_test, y_pred, cv_score)
    r2_list.append(r2)
    mae_list.append(mae)
    mse_list.append(mse)
print()
print('Total')
print('R2 : %.4f' % (float(np.mean(r2_list))))
print('StD : %.4f' % (float(np.std(r2_list))))
print('MAE : %.4f' % (float(np.mean(mae_list))))
print('MSE : %.4f' % (float(np.mean(mse_list))))